In [2]:
%%bash 
pip install arch
pip install numpy
pip install pandas
pip install seaborn
pip install mlxtend

In [4]:
display(A)

,Date,Location,festival,rooms,year,rain,Bioindicator,Coliform,total,week,Bioindicator UV+
0,2016-06-21,MRD,before,3069.600000,16,0.0,66.666667,466.666667,900.000000,1,NaN
1,2016-06-21,SVT,before,3069.600000,16,0.0,33.333333,33.333333,133.333333,1,NaN
2,2016-06-21,VNX,before,3069.600000,16,0.0,0.000000,0.000000,66.666667,1,NaN
3,2016-06-28,MRD,before,3069.600000,16,1.6,8.333333,25.000000,491.666667,2,NaN
4,2016-06-28,SVT,before,3069.600000,16,1.6,0.000000,8.333333,391.666667,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
67,2020-07-22,SVT,before,8058.967742,20,7.8,100.000000,1000.000000,6066.666667,7,NaN
68,2020-07-22,VNX,before,8058.967742,20,7.8,0.000000,200.000000,666.666667,7,NaN
69,2020-08-06,MRD,before,7890.741935,20,1.8,0.000000,233.333333,3100.000000,8,NaN
70,2020-08-06,SVT,before,7890.741935,20,1.8,166.666667,633.333333,3366.666667,8,NaN


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import os
import seaborn as sns
from scipy import stats
from mlxtend.evaluate import permutation_test

In [3]:
#A=pd.read_pickle("data/allyears-avg_plate.pkl")
A=pd.read_pickle("data/allyears-avg_plate.pkl")

In [114]:
# # Permutation test for combined off-festival vs festival period
# p_value = permutation_test( A.loc[A['festival'] == 'during', 'Bioindicator'] , 
#                             A.loc[A['festival'] != 'during', 'Bioindicator'] , 
#                            method='approximate',
#                            num_rounds=10000,
#                            seed=0)
# print(p_value)

In [113]:
# for c in ['Bioindicator','Coliform']:
#     p1 = permutation_test( A.loc[A['year'] == 16 , c] , 
#                                 A.loc[A['year'] == 17, c] , 
#                                method='approximate',
#                                num_rounds=10000,
#                                seed=0)
#     p2 = permutation_test( A.loc[A['year'] == 16 , c] , 
#                                 A.loc[A['year'] == 20, c] , 
#                                method='approximate',
#                                num_rounds=10000,
#                                seed=0)
#     p3 = permutation_test( A.loc[A['year'] == 17 , c] , 
#                                 A.loc[A['year'] == 20, c] , 
#                                method='approximate',
#                                num_rounds=10000,
#                                seed=0)
#     print(f"16vs17 = {p1}, 16vs20 = {p2}, 17vs20 = {p3} on {c}")

In [112]:
# p_value = permutation_test( A.loc[(A['year']==16) & (A['festival'] == 'during'), 'Bioindicator'] , 
#                             A.loc[(A['year']==17) & (A['festival'] == 'during'), 'Bioindicator'], 
#                            method='approximate',
#                            num_rounds=10000,
#                            seed=0)
# print(p_value)
#0.35326467353264673 BUV+

In [111]:
# p_value = permutation_test( A.loc[(A['year']==16) & (A['festival'] == 'during'), 'Bioindicator'] , 
#                             A.loc[(A['year']==16) & (A['festival'] != 'during'), 'Bioindicator'], 
#                            method='approximate',
#                            num_rounds=10000,
#                            seed=0)
# print(p_value)

In [110]:
# p_value = permutation_test( A.loc[(A['year']==17) & (A['festival'] == 'during'), 'Bioindicator'] , 
#                             A.loc[(A['year']==17) & (A['festival'] != 'during'), 'Bioindicator'], 
#                            method='approximate',
#                            num_rounds=10000,
#                            seed=0)
# print(p_value)

## Permutation test figure 4B

In [54]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [68]:
p_value = permutation_test( A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'before'), 'Bioindicator'] , 
                            A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'during'), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

0.00019998000199980003


In [69]:
p_value = permutation_test( A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'during'), 'Bioindicator'] , 
                            A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'after'), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

0.035996400359964


In [70]:
p_value = permutation_test( A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'before'), 'Bioindicator'] , 
                            A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'after'), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

0.10208979102089791


In [76]:
p_value = permutation_test( A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'before'), 'Bioindicator'] , 
                            A.loc[(A['year']==20) & (A['festival'] == 'No festival'), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(format(p_value, ".5f"))

/Users/kato/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kato/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.00010


In [77]:
p_value = permutation_test( A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'during'), 'Bioindicator'] , 
                            A.loc[((A['year']==20) & (A['festival'] == 'No festival')), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(format(p_value, ".4f"))

/Users/kato/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kato/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.0001


In [78]:
p_value = permutation_test( A.loc[((A['year']==16) | (A['year']==17)) & (A['festival'] == 'after'), 'Bioindicator'] , 
                            A.loc[((A['year']==20) & (A['festival'] == 'No festival')), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(format(p_value, ".4f"))

0.0001


/Users/kato/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kato/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Old Correlations (See below)

In [109]:
# main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
# A[main_vars].corr(method ='spearman')

In [107]:
# def append_vars(ds, cols):
#     v=ds[cols]
#     X=ds.drop(columns=cols)
#     return pd.concat([X.assign(cfu=ds[col], name=ds['year'].apply(lambda y: f"{col} {y}")) for col in cols]).dropna()

In [106]:
# A2=append_vars(A, ['Bioindicator', 'Bioindicator UV+', 'Coliform', 'total'])

In [105]:
# A.query('year==16')[main_vars].corr(method ='spearman')

In [104]:
# A.query('year==17')[main_vars].corr(method ='spearman')

In [103]:
# A.query('year==20')[main_vars].corr(method ='spearman')

In [100]:
# A[main_vars].corr(method ='spearman') 

In [99]:
# A2[['cfu', 'rooms', 'rain']].corr(method ='spearman')

In [98]:
# def bootstrap_ci(df, variable, classes, repetitions = 1000, alpha = 0.05, random_state=None): 
    
#     df = df[[variable, classes]]
#     bootstrap_sample_size = len(df) 
    
#     mean_diffs = []
#     for i in range(repetitions):
#             bootstrap_sample = df.sample(n = bootstrap_sample_size, replace = True, random_state = random_state)
#             mean_diff = bootstrap_sample.groupby(classes).mean().iloc[1,0] - bootstrap_sample.groupby(classes).mean().iloc[0,0]
#             mean_diffs.append(mean_diff)
#     # confidence interval
#     left = np.percentile(mean_diffs, alpha/2*100)
#     right = np.percentile(mean_diffs, 100-alpha/2*100)
#     # point estimate
#     point_est = df.groupby(classes).mean().iloc[1,0] - df.groupby(classes).mean().iloc[0,0]
#     print('Point estimate of difference between means:', round(point_est,2))
#     print((1-alpha)*100,'%','confidence interval for the difference between means:', (round(left,2), round(right,2)))


In [97]:
# import arch.data.frenchdata
# from arch.bootstrap import StationaryBootstrap,IndependentSamplesBootstrap

In [96]:
# def foi(X):
#     return X.corr(method ='spearman').iloc[0,1] #['cfu']['rain']
    
# bs = StationaryBootstrap(12, A[['Bioindicator','rain']])
# results = bs.apply(foi, 2500)
# SR = pd.DataFrame(results[:, -1:], columns=["SR"])
# fig = SR.hist(bins=40)


In [95]:
# A[['Bioindicator','rain']].corr(method='spearman').iloc[0,1]

In [94]:
# l = SR.SR

# total_size = len(SR.SR)
# true = np.sum(l <0.12)

# true/total_size

In [93]:
# def foi(X):
#     return X[['cfu', 'rooms', 'rain']].corr(method ='spearman')['cfu']['rain']
    
# bs = StationaryBootstrap(12, A2)
# results = bs.apply(foi, 2500)
# SR = pd.DataFrame(results[:, -1:], columns=["SR"])
# fig = SR.hist(bins=40)

In [92]:
# ci = bs.conf_int(foi, 1000, method="percentile", reuse=True)
# ci

## Bootstrap

In [91]:
# def boot_matrix(z, B):
#     """Bootstrap sample
    
#     Returns all bootstrap samples in a matrix"""
    
#     n = len(z)  # sample size
#     idz = np.random.randint(0, n, size=(B, n))  # indices to pick for all boostrap samples
#     return z[idz]

# def bootstrap_mean(x, B=10000, alpha=0.05, plot=False):
#     """Bootstrap standard error and (1-alpha)*100% c.i. for the population mean
    
#     Returns bootstrapped standard error and different types of confidence intervals"""
   
#     # Deterministic things
#     n = len(x)  # sample size
#     orig = x.mean()  # sample mean
#     se_mean = x.std()/np.sqrt(n) # standard error of the mean
#     qt = stats.t.ppf(q=1 - alpha/2, df=n - 1) # Student quantile
    
#     # Generate boostrap distribution of sample mean
#     xboot = boot_matrix(x, B=B)
#     sampling_distribution = xboot.mean(axis=1)
   
#    # Standard error and sample quantiles
#     se_mean_boot = sampling_distribution.std()
#     quantile_boot = np.percentile(sampling_distribution, q=(100*alpha/2, 100*(1-alpha/2)))
 
#     # RESULTS
#     print("Estimated mean:", orig)
#     print("Classic standard error:", se_mean)
#     print("Classic student c.i.:", orig + np.array([-qt, qt])*se_mean)
#     print("\nBootstrap results:")
#     print("Standard error:", se_mean_boot)
#     print("t-type c.i.:", orig + np.array([-qt, qt])*se_mean_boot)
#     print("Percentile c.i.:", quantile_boot)
#     print("Basic c.i.:", 2*orig - quantile_boot[::-1])

#     if plot:
#         plt.hist(sampling_distribution, bins="fd")

In [90]:
# def mean_diff(x, y):
#     return x.mean() - y.mean()

# bs = IndependentSamplesBootstrap(A.loc[A['year'] == 16], A.loc[A['year'] == 20])
# print(bs.conf_int(mean_diff, method="studentized"))


## Carlos correlations and p-value (Pearson)


Taken from a answer here: https://stackoverflow.com/questions/25571882/pandas-columns-correlation-with-statistical-significance

In [89]:
# from scipy.stats import pearsonr
# import numpy as np

# def calculate_pvalues(df):
#     df = df.dropna()._get_numeric_data()
#     dfcols = pd.DataFrame(columns=df.columns)
#     pvalues = dfcols.transpose().join(dfcols, how='outer')
#     for r in df.columns:
#         for c in df.columns:
#             pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
#     return pvalues


# main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
# p_df = A[main_vars]

# rho = p_df.corr()
# pval = p_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p

In [87]:
# p16_df= A.query('year==16')[main_vars]

# rho = p16_df.corr()
# pval = p16_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p

In [86]:
# p17_df= A.query('year==17')[main_vars]

# rho = p17_df.corr()
# pval = p17_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p

In [85]:
# p20_df= A.query('year==20')[main_vars]

# rho = p20_df.corr()
# pval = p20_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p

## Correlation with Spearman Method 24 Hours 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

In [14]:
A=pd.read_pickle("data/allyears.pkl")

In [15]:
from scipy.stats import spearmanr
import numpy as np


def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues


main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
p_df = A[main_vars]

rho = p_df.corr()
pval = p_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
rho.round(2).astype(str) + p

,Bioindicator,Coliform,Bioindicator UV+,total,rain,rooms
Bioindicator,1.0***,0.09***,0.99***,0.16***,-0.03*,0.23***
Coliform,0.09***,1.0***,0.46***,0.68***,-0.12,0.04***
Bioindicator UV+,0.99***,0.46***,1.0***,0.42***,-0.3,0.31*
total,0.16***,0.68***,0.42***,1.0***,-0.01,0.25***
rain,-0.03*,-0.12,-0.3,-0.01,1.0***,0.25***
rooms,0.23***,0.04***,0.31*,0.25***,0.25***,1.0***


In [16]:
main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
p_df = A[main_vars]

rho = p_df.corr()
pval = p_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join([' p-value: ', str(format(x, ".5f"))]))
rho.round(2).astype(str) + p

,Bioindicator,Coliform,Bioindicator UV+,total,rain,rooms
Bioindicator,1.0 p-value: 0.00000,0.09 p-value: 0.00001,0.99 p-value: 0.00000,0.16 p-value: 0.00000,-0.03 p-value: 0.06981,0.23 p-value: 0.00002
Coliform,0.09 p-value: 0.00001,1.0 p-value: 0.00000,0.46 p-value: 0.00004,0.68 p-value: 0.00000,-0.12 p-value: 0.66865,0.04 p-value: 0.00239
Bioindicator UV+,0.99 p-value: 0.00000,0.46 p-value: 0.00004,1.0 p-value: 0.00000,0.42 p-value: 0.00000,-0.3 p-value: 0.12247,0.31 p-value: 0.05533
total,0.16 p-value: 0.00000,0.68 p-value: 0.00000,0.42 p-value: 0.00000,1.0 p-value: 0.00000,-0.01 p-value: 0.43823,0.25 p-value: 0.00238
rain,-0.03 p-value: 0.06981,-0.12 p-value: 0.66865,-0.3 p-value: 0.12247,-0.01 p-value: 0.43823,1.0 p-value: 0.00000,0.25 p-value: 0.00000
rooms,0.23 p-value: 0.00002,0.04 p-value: 0.00239,0.31 p-value: 0.05533,0.25 p-value: 0.00238,0.25 p-value: 0.00000,1.0 p-value: 0.00000


## Correlation with Spearman Method cumulative 48 Hours

In [8]:
A=pd.read_pickle("data/allyears_48h_cumulative.pkl")

In [9]:
from scipy.stats import spearmanr
import numpy as np


def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues


main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
p_df = A[main_vars]

rho = p_df.corr()
pval = p_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
rho.round(2).astype(str) + p

,Bioindicator,Coliform,Bioindicator UV+,total,rain,rooms
Bioindicator,1.0***,0.09***,0.99***,0.16***,-0.13,0.16***
Coliform,0.09***,1.0***,0.46***,0.68***,-0.06,-0.2***
Bioindicator UV+,0.99***,0.46***,1.0***,0.42***,0.17**,-0.03
total,0.16***,0.68***,0.42***,1.0***,-0.09**,-0.27***
rain,-0.13,-0.06,0.17**,-0.09**,1.0***,-0.43***
rooms,0.16***,-0.2***,-0.03,-0.27***,-0.43***,1.0***


In [10]:
main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
p_df = A[main_vars]

rho = p_df.corr()
pval = p_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join([' p-value: ', str(format(x, ".5f"))]))
rho.round(2).astype(str) + p

,Bioindicator,Coliform,Bioindicator UV+,total,rain,rooms
Bioindicator,1.0 p-value: 0.00000,0.09 p-value: 0.00001,0.99 p-value: 0.00000,0.16 p-value: 0.00000,-0.13 p-value: 0.65244,0.16 p-value: 0.00314
Coliform,0.09 p-value: 0.00001,1.0 p-value: 0.00000,0.46 p-value: 0.00004,0.68 p-value: 0.00000,-0.06 p-value: 0.29490,-0.2 p-value: 0.00002
Bioindicator UV+,0.99 p-value: 0.00000,0.46 p-value: 0.00004,1.0 p-value: 0.00000,0.42 p-value: 0.00000,0.17 p-value: 0.02381,-0.03 p-value: 0.70203
total,0.16 p-value: 0.00000,0.68 p-value: 0.00000,0.42 p-value: 0.00000,1.0 p-value: 0.00000,-0.09 p-value: 0.04796,-0.27 p-value: 0.00000
rain,-0.13 p-value: 0.65244,-0.06 p-value: 0.29490,0.17 p-value: 0.02381,-0.09 p-value: 0.04796,1.0 p-value: 0.00000,-0.43 p-value: 0.00951
rooms,0.16 p-value: 0.00314,-0.2 p-value: 0.00002,-0.03 p-value: 0.70203,-0.27 p-value: 0.00000,-0.43 p-value: 0.00951,1.0 p-value: 0.00000


## Correlation with Spearman Method cumulative 72 Hours

In [11]:
A=pd.read_pickle("data/allyears_72h_cumulative.pkl")

In [12]:
from scipy.stats import spearmanr
import numpy as np


def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues


main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
p_df = A[main_vars]

rho = p_df.corr()
pval = p_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
rho.round(2).astype(str) + p

,Bioindicator,Coliform,Bioindicator UV+,total,rain,rooms
Bioindicator,1.0***,0.09***,0.99***,0.16***,-0.16,0.16***
Coliform,0.09***,1.0***,0.46***,0.68***,-0.12,-0.2***
Bioindicator UV+,0.99***,0.46***,1.0***,0.42***,0.13***,-0.03
total,0.16***,0.68***,0.42***,1.0***,-0.12,-0.27***
rain,-0.16,-0.12,0.13***,-0.12,1.0***,-0.41***
rooms,0.16***,-0.2***,-0.03,-0.27***,-0.41***,1.0***


In [13]:
main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
p_df = A[main_vars]

rho = p_df.corr()
pval = p_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join([' p-value: ', str(format(x, ".5f"))]))
rho.round(2).astype(str) + p

,Bioindicator,Coliform,Bioindicator UV+,total,rain,rooms
Bioindicator,1.0 p-value: 0.00000,0.09 p-value: 0.00001,0.99 p-value: 0.00000,0.16 p-value: 0.00000,-0.16 p-value: 0.68576,0.16 p-value: 0.00314
Coliform,0.09 p-value: 0.00001,1.0 p-value: 0.00000,0.46 p-value: 0.00004,0.68 p-value: 0.00000,-0.12 p-value: 0.76425,-0.2 p-value: 0.00002
Bioindicator UV+,0.99 p-value: 0.00000,0.46 p-value: 0.00004,1.0 p-value: 0.00000,0.42 p-value: 0.00000,0.13 p-value: 0.00963,-0.03 p-value: 0.70203
total,0.16 p-value: 0.00000,0.68 p-value: 0.00000,0.42 p-value: 0.00000,1.0 p-value: 0.00000,-0.12 p-value: 0.63589,-0.27 p-value: 0.00000
rain,-0.16 p-value: 0.68576,-0.12 p-value: 0.76425,0.13 p-value: 0.00963,-0.12 p-value: 0.63589,1.0 p-value: 0.00000,-0.41 p-value: 0.00658
rooms,0.16 p-value: 0.00314,-0.2 p-value: 0.00002,-0.03 p-value: 0.70203,-0.27 p-value: 0.00000,-0.41 p-value: 0.00658,1.0 p-value: 0.00000


In [142]:
p = pval.applymap(lambda x: ''.join(str(format(x, ".5f"))))

rho.to_csv('tables/Spearman_correlation.csv')
p.to_csv('tables/Spearman_p_values.csv')
#with pd.ExcelWriter('tables/Spearman_correlation.xlsx') as writer:  
#    rho.to_(writer, sheet_name='Correlations')
#    p.to_excel(writer, sheet_name='P-Value')

In [79]:
# p16_df= A.query('year==16')[main_vars]

# rho = p16_df.corr()
# pval = p16_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p
# plt.show()

# rho = p16_df.corr()
# pval = p16_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join([' p-value: ', str(format(x, ".5f"))]))
# rho.round(2).astype(str) + p

In [80]:
# p17_df= A.query('year==17')[main_vars]

# rho = p17_df.corr()
# pval = p17_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p
# plt.show()

# rho = p17_df.corr()
# pval = p17_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join([' p-value: ', str(format(x, ".5f"))]))
# rho.round(2).astype(str) + p

In [81]:
# p1617_df= A.query('(year==16) | (year == 17)')[main_vars]

# rho = p1617_df.corr()
# pval = p1617_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join([' p-value: ', str(format(x, ".5f"))]))
# rho.round(2).astype(str) + p

In [82]:
# p20_df= A.query('year==20')[main_vars]

# rho = p20_df.corr()
# pval = p20_df.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p

## Kendals Tau

In [83]:
# from scipy.stats import kendalltau
# import numpy as np


# def calculate_pvalues(df):
#     df = df.dropna()._get_numeric_data()
#     dfcols = pd.DataFrame(columns=df.columns)
#     pvalues = dfcols.transpose().join(dfcols, how='outer')
#     for r in df.columns:
#         for c in df.columns:
#             pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
#     return pvalues


# main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
# p_df = A[main_vars]

# rho = p_df.corr()
# pval = p_df.corr(method=lambda x, y: kendalltau(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
# rho.round(2).astype(str) + p

In [84]:
# main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
# p_df = A[main_vars]

# rho = p_df.corr()
# pval = p_df.corr(method=lambda x, y: kendalltau(x, y)[1]) - np.eye(*rho.shape)
# p = pval.applymap(lambda x: ''.join([' p-value: ', str(format(x, ".5f"))]))
# rho.round(2).astype(str) + p